In [ ]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=fd6695d6bff7c5e6cb022355aa29342e143ef2fb7c8aeb5409c37719ce6fd5c6
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
pip install pandas

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count

# Membuat Spark session
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

# Data dan kolom untuk DataFrame
data = [('Wahyu', 'Sales', 3000),
        ('Fariz', 'Advokasi', 4600),
        ('Popo', 'Seketaris', 4100),
        ('Remon', 'Advokasi', 3600),
        ('David', 'Finance', 3100),
        ('Mada', 'Sales', 3000),
        ('Aji', 'Finance', 3400),
        ]
columns = ['Nama Karyawan', 'Devisi', 'Gaji']


df = spark.createDataFrame(data, schema=columns)


print( "DataFrame Awal")
df.show()

# # Menghitung jumlah employee di setiap department
# print("Jumlah Karyawan di Setiap departemen")
# df.groupBy("Devisi").agg(count("Nama Karyawan").alias("Jumlah Karyawan")).show()

# # Menghitung total salary per department
# print( "Jumlah Gaji Setiap Departement")
# df.groupBy("Devisi").agg(sum("Gaji").alias("Total Gaji")).show()

# # Menghitung rata-rata salary per department
# print( "Rata rata gaji setiap departemen")
# df.groupBy("Devisi").avg("Gaji").show()

# # Menampilkan kolom dengan kondisi
# print( "Gaji yang lebih besar dari 3500")
# df.filter(col("Gaji") > 3500).show()

# print( "Gaji yang lebih kecil dari 3500")
# df.filter(col("Gaji") < 3500).show()


DataFrame Awal
+-------------+---------+----+
|Nama Karyawan|   Devisi|Gaji|
+-------------+---------+----+
|        Wahyu|    Sales|3000|
|        Fariz| Advokasi|4600|
|         Popo|Seketaris|4100|
|        Remon| Advokasi|3600|
|        David|  Finance|3100|
|         Mada|    Sales|3000|
|          Aji|  Finance|3400|
+-------------+---------+----+



In [ ]:
print( "DataFrame Awal")
df.select('Nama Karyawan', 'Gaji').show()

print( "Gaji yang lebih besar dari 3000")
df.filter(df['Gaji'] > 3000).show()

print( "Gaji yang lebih kecil dari 3000")
df.filter(df['Gaji'] < 3500).show()

print( "Jumlah Gaji Setiap Departement")
df.groupBy('Devisi').agg(avg('Gaji').alias("Gaji")).show()

DataFrame Awal
+-------------+----+
|Nama Karyawan|Gaji|
+-------------+----+
|        Wahyu|3000|
|        Fariz|4600|
|         Popo|4100|
|        Remon|3600|
|        David|3100|
|         Mada|3000|
|          Aji|3400|
+-------------+----+

Gaji yang lebih besar dari 3000
+-------------+---------+----+
|Nama Karyawan|   Devisi|Gaji|
+-------------+---------+----+
|        Fariz| Advokasi|4600|
|         Popo|Seketaris|4100|
|        Remon| Advokasi|3600|
|        David|  Finance|3100|
|          Aji|  Finance|3400|
+-------------+---------+----+

Gaji yang lebih kecil dari 3000
+-------------+-------+----+
|Nama Karyawan| Devisi|Gaji|
+-------------+-------+----+
|        Wahyu|  Sales|3000|
|        David|Finance|3100|
|         Mada|  Sales|3000|
|          Aji|Finance|3400|
+-------------+-------+----+

Jumlah Gaji Setiap Departement
+---------+------+
|   Devisi|  Gaji|
+---------+------+
|Seketaris|4100.0|
|    Sales|3000.0|
| Advokasi|4100.0|
|  Finance|3250.0|
+---------+-

In [ ]:
df = df.withColumn('Bonus Gaji', df['Gaji'] * 0.1)
df = df.withColumn('Total', df['Gaji'] + df['Bonus Gaji'])
df.show()

+-------------+---------+----+----------+------+
|Nama Karyawan|   Devisi|Gaji|Bonus Gaji| Total|
+-------------+---------+----+----------+------+
|        Wahyu|    Sales|3000|     300.0|3300.0|
|        Fariz| Advokasi|4600|     460.0|5060.0|
|         Popo|Seketaris|4100|     410.0|4510.0|
|        Remon| Advokasi|3600|     360.0|3960.0|
|        David|  Finance|3100|     310.0|3410.0|
|         Mada|    Sales|3000|     300.0|3300.0|
|          Aji|  Finance|3400|     340.0|3740.0|
+-------------+---------+----+----------+------+



In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Devisi').orderBy('Gaji')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+-------------+---------+----+----------+------+----+
|Nama Karyawan|   Devisi|Gaji|Bonus Gaji| Total|Rank|
+-------------+---------+----+----------+------+----+
|        Remon| Advokasi|3600|     360.0|3960.0|   1|
|        Fariz| Advokasi|4600|     460.0|5060.0|   2|
|        David|  Finance|3100|     310.0|3410.0|   1|
|          Aji|  Finance|3400|     340.0|3740.0|   2|
|        Wahyu|    Sales|3000|     300.0|3300.0|   1|
|         Mada|    Sales|3000|     300.0|3300.0|   1|
|         Popo|Seketaris|4100|     410.0|4510.0|   1|
+-------------+---------+----+----------+------+----+

